# Configuration

In [1]:
# Import libraries
import pandas               as pd
import datetime             as dt
import numpy                as np
import os                   as os
import plotly.graph_objects as go
import requests             as requests

from PIL                   import Image
from datetime              import date
from datetime              import timedelta
from amberdata_derivatives import AmberdataDerivatives

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Retrieve API key
AD_api_key = os.getenv('API_KEY') #INPUT YOUR AD KEY HERE

# Amberdata Colors & Logo
AD_COLORS    = ['#231342', '#882c91', '#e00d8e', '#4c61ab', '#87ddf3']
AD_COLORS_FB = ['#4c61ab','#e00d8e','#89bed8','#882c91','#f16623']
AD_COLORS_3  = ['#1c3664', '#89bed8', '#f16623']

AD_LOGO      = Image.open("amberdata_logo_bug_color_100p.png")

# Amberdata SDK

In [2]:
AD_client = AmberdataDerivatives(api_key=AD_api_key)

headers = {
    "accept"          : "application/json",
    "Accept-Encoding" : "gzip",
    "x-api-key"       : AD_api_key
}

# Helper Functions

In [3]:
def svi_raw(params, k):
    '''
    This function is the raw SVI formula.
    Return volatility (varianace squared)
    
    params: list of 5 SVI parameters
    k: list of log moneyness values to fit 
    '''    
    a, b, sigma, rho, m = params
    return np.sqrt(a + b * (rho * (k - m) + np.sqrt((k - m) ** 2 + sigma ** 2)))

def apply_svi(row):
    params = [row['sviA'], row['sviB'], row['sviSigma'], row['sviRho'], row['sviM']]
    k      = row['moneyness']
    return svi_raw(params, k) * 100

In [4]:
def fecth_svi_historical(exchange, currency):
    try:
        base_url = "https://api.amberdata.com"
        endpoint = f"/markets/derivatives/analytics/volatility/svi-historical?currency={currency}&exchange={exchange}&timeInterval=minute"
        url      = f"{base_url}{endpoint}"
        response = requests.get(url, headers=headers)
        response.raise_for_status() 

        data = response.json()
        svi  = pd.json_normalize(data['payload']['data'])

        svi['timestamp'] = pd.to_datetime(svi['timestamp'], unit='ms')
        svi['atmIv']     = (svi_raw([svi['sviA'], svi['sviB'], svi['sviSigma'], svi['sviRho'], svi['sviM']], 0)) * 100
        svi.sort_values('timestamp', ascending=False, inplace=True)

        return svi
    except requests.RequestException as e:
        print (f"Erorr: {e}")

In [5]:
def fetch_hifi(exchange, currency, hours_ago):
    svi_timestamp_last     =  pd.to_datetime(svi_ts['timestamp'].unique()[0]).strftime('%Y-%m-%dT%H:%M:%S.%f')
    svi_timestamp_last_end = (pd.to_datetime(svi_ts['timestamp'].unique()[0]) + timedelta(minutes=1)).strftime('%Y-%m-%dT%H:%M:%S')

    data = AD_client.get_volatility_level_1_quotes(
        exchange  = exchange,
        currency  = currency,
        startDate = svi_timestamp_last,
        endDate   = svi_timestamp_last_end
    )

    hifi                            = pd.json_normalize(data['payload']['data'])
    hifi['timestamp']               = pd.to_datetime(hifi['timestamp']          ).dt.tz_localize(None)
    hifi['expirationTimestamp']     = pd.to_datetime(hifi['expirationTimestamp']).dt.tz_localize(None)
    hifi['expirationTimestampDate'] = pd.to_datetime(hifi['expirationTimestamp'].dt.date).dt.tz_localize(None)

    hifi.loc[(hifi['strike'] <  hifi['underlyingPrice']) & (hifi['putCall'] == 'P'), 'moneyness'] = 'OTM'
    hifi.loc[(hifi['strike'] >= hifi['underlyingPrice']) & (hifi['putCall'] == 'P'), 'moneyness'] = 'ITM'
    hifi.loc[(hifi['strike'] <  hifi['underlyingPrice']) & (hifi['putCall'] == 'C'), 'moneyness'] = 'ITM'
    hifi.loc[(hifi['strike'] >= hifi['underlyingPrice']) & (hifi['putCall'] == 'C'), 'moneyness'] = 'OTM'

    return hifi[hifi['moneyness']=='OTM']

In [6]:
def generate_equidistant_strikes(hifi_merged_current, step_size):
    df = pd.DataFrame()

    for exp in hifi_merged_current.expirationTimestamp_hifi.unique():
        hifi_exp = hifi_merged_current[hifi_merged_current['expirationTimestamp_hifi'] == exp]
        strikes  = hifi_exp.sort_values('strike')['strike'].unique()

        # Definisci il minimo e il massimo degli strike
        strike_min = min(strikes)
        strike_max = max(strikes)

        # Genera strike equidistanti usando np.arange()
        new_strikes = np.arange(strike_min, strike_max + step_size, step_size)  # Aggiungi step_size per includere il max

        # Crea il DataFrame per gli strike generati
        df_strikes = pd.DataFrame(new_strikes, columns=['strike'])
        df_strikes['expirationTimestamp'] = exp

        # Concatena i risultati al DataFrame finale
        df = pd.concat([df, df_strikes])

    return df

# Retrieve SVI

In [7]:
currency = 'BTC'
exchange = 'deribit'

In [8]:
svi                        = fecth_svi_historical(exchange, currency)
svi['timestamp']           = pd.to_datetime(svi['timestamp'], unit='ms')
svi['expirationTimestamp'] = pd.to_datetime(svi['expirationTimestamp']).dt.tz_localize(None)
svi['underlyingPrice']     = svi['indexPrice'] + svi['forwardDifference']
svi.sort_values('timestamp', ascending=False, inplace=True)

svi_ts = svi[svi['timestamp'] == svi['timestamp'].max()-timedelta(hours=0)]

svi_ts

,currency,daysToExpiration,expirationTimestamp,forwardDifference,indexPrice,sviA,sviB,sviM,sviRho,sviSigma,timestamp,atmIv,underlyingPrice
0,BTC,1.06,2024-11-14,27.648438,86598.89,-1.339611,13.652679,-0.056911,-0.358936,0.141319,2024-11-13 06:30:00,67.931157,86626.538437
3,BTC,16.06,2024-11-29,564.515625,86598.89,-1.519727,3.405857,-0.108365,-0.200618,0.547454,2024-11-13 06:30:00,55.403828,87163.405625
4,BTC,44.06,2024-12-27,1474.390625,86598.89,-0.742481,1.563526,-0.100509,-0.160213,0.672158,2024-11-13 06:30:00,54.310499,88073.280625
5,BTC,79.06,2025-01-31,2451.429688,86598.89,-0.090174,0.846603,-0.333426,-0.438553,0.503705,2024-11-13 06:30:00,54.537420,89050.319687
6,BTC,135.06,2025-03-28,3994.171875,86598.89,-0.834762,0.928057,-0.334448,-0.215318,1.242872,2024-11-13 06:30:00,54.119574,90593.061875
7,BTC,226.06,2025-06-27,6449.765625,86598.89,-0.498117,0.599533,-0.327342,-0.189769,1.344493,2024-11-13 06:30:00,54.245266,93048.655625
8,BTC,317.06,2025-09-26,8634.742188,86598.89,0.064127,0.299875,-0.452099,-0.332993,0.813112,2024-11-13 06:30:00,54.586601,95233.632187
1,BTC,2.06,2024-11-15,44.398438,86598.89,-0.219569,6.317711,-0.034087,-0.264439,0.102349,2024-11-13 06:30:00,63.640641,86643.288437
2,BTC,9.06,2024-11-22,292.398438,86598.89,-0.562798,3.303066,-0.085524,-0.275817,0.277940,2024-11-13 06:30:00,56.552640,86891.288437


# Retrieve Level-1-Quotes

In [9]:
hifi_current        = fetch_hifi(exchange, currency, hours_ago=0)
hifi_merged_current = hifi_current.merge(svi_ts, left_on='expirationTimestampDate', right_on='expirationTimestamp', suffixes=('_hifi','_svi'))

hifi_merged_current

,ask,askIv,askVolume,bid,bidIv,bidVolume,currency_hifi,delta,exchange,exchangeTimestamp,...,forwardDifference,indexPrice_svi,sviA,sviB,sviM,sviRho,sviSigma,timestamp_svi,atmIv,underlyingPrice_svi
0,0.0002,132.59,7.0,0.0000,0.00,0.0,BTC,0.00067,deribit,2024-11-13T06:30:00.697Z,...,27.648438,86598.89,-1.339611,13.652679,-0.056911,-0.358936,0.141319,2024-11-13 06:30:00,67.931157,86626.538437
1,0.0001,116.46,1.6,0.0000,0.00,0.0,BTC,0.00103,deribit,2024-11-13T06:30:00.697Z,...,27.648438,86598.89,-1.339611,13.652679,-0.056911,-0.358936,0.141319,2024-11-13 06:30:00,67.931157,86626.538437
2,0.0003,125.78,3.5,0.0000,0.00,0.0,BTC,0.00156,deribit,2024-11-13T06:30:00.697Z,...,27.648438,86598.89,-1.339611,13.652679,-0.056911,-0.358936,0.141319,2024-11-13 06:30:00,67.931157,86626.538437
3,0.0004,123.57,40.3,0.0000,0.00,0.0,BTC,0.00269,deribit,2024-11-13T06:30:00.697Z,...,27.648438,86598.89,-1.339611,13.652679,-0.056911,-0.358936,0.141319,2024-11-13 06:30:00,67.931157,86626.538437
4,0.0002,105.83,0.1,0.0000,0.00,0.0,BTC,0.00482,deribit,2024-11-13T06:30:00.697Z,...,27.648438,86598.89,-1.339611,13.652679,-0.056911,-0.358936,0.141319,2024-11-13 06:30:00,67.931157,86626.538437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,0.0315,57.40,16.3,0.0280,55.19,16.0,BTC,-0.09533,deribit,2024-11-13T06:30:03.793Z,...,8634.742188,86598.89,0.064127,0.299875,-0.452099,-0.332993,0.813112,2024-11-13 06:30:00,54.586601,95233.632187
430,0.0225,58.55,17.8,0.0195,56.16,18.9,BTC,-0.07078,deribit,2024-11-13T06:30:00.877Z,...,8634.742188,86598.89,0.064127,0.299875,-0.452099,-0.332993,0.813112,2024-11-13 06:30:00,54.586601,95233.632187
431,0.0115,63.15,36.3,0.0090,59.74,56.5,BTC,-0.03610,deribit,2024-11-13T06:30:03.793Z,...,8634.742188,86598.89,0.064127,0.299875,-0.452099,-0.332993,0.813112,2024-11-13 06:30:00,54.586601,95233.632187
432,0.0055,70.22,40.1,0.0035,64.80,2.4,BTC,-0.01584,deribit,2024-11-13T06:30:03.793Z,...,8634.742188,86598.89,0.064127,0.299875,-0.452099,-0.332993,0.813112,2024-11-13 06:30:00,54.586601,95233.632187


# Generate Strikes

In [10]:
df = generate_equidistant_strikes(hifi_merged_current, step_size=50)
df

,strike,expirationTimestamp
0,72000,2024-11-14 08:00:00
1,72050,2024-11-14 08:00:00
2,72100,2024-11-14 08:00:00
3,72150,2024-11-14 08:00:00
4,72200,2024-11-14 08:00:00
...,...,...
4796,259800,2025-09-26 08:00:00
4797,259850,2025-09-26 08:00:00
4798,259900,2025-09-26 08:00:00
4799,259950,2025-09-26 08:00:00


# Generate Curves

In [11]:
final_current = df.merge(hifi_merged_current, left_on=['strike','expirationTimestamp'], right_on=['strike','expirationTimestamp_hifi'],how='left').ffill()    

final_current['moneyness'] = np.log(final_current['strike']/final_current['underlyingPrice_svi'])
final_current['sviIv']     = final_current.apply(apply_svi, axis=1)

final_current

,strike,expirationTimestamp,ask,askIv,askVolume,bid,bidIv,bidVolume,currency_hifi,delta,...,indexPrice_svi,sviA,sviB,sviM,sviRho,sviSigma,timestamp_svi,atmIv,underlyingPrice_svi,sviIv
0,72000,2024-11-14 08:00:00,0.0002,143.66,1.0,0.0000,0.00,0.0,BTC,-0.00529,...,86598.89,-1.339611,13.652679,-0.056911,-0.358936,0.141319,2024-11-13 06:30:00,67.931157,86626.538437,137.521473
1,72050,2024-11-14 08:00:00,0.0002,143.66,1.0,0.0000,0.00,0.0,BTC,-0.00529,...,86598.89,-1.339611,13.652679,-0.056911,-0.358936,0.141319,2024-11-13 06:30:00,67.931157,86626.538437,137.166315
2,72100,2024-11-14 08:00:00,0.0002,143.66,1.0,0.0000,0.00,0.0,BTC,-0.00529,...,86598.89,-1.339611,13.652679,-0.056911,-0.358936,0.141319,2024-11-13 06:30:00,67.931157,86626.538437,136.811180
3,72150,2024-11-14 08:00:00,0.0002,143.66,1.0,0.0000,0.00,0.0,BTC,-0.00529,...,86598.89,-1.339611,13.652679,-0.056911,-0.358936,0.141319,2024-11-13 06:30:00,67.931157,86626.538437,136.456070
4,72200,2024-11-14 08:00:00,0.0002,143.66,1.0,0.0000,0.00,0.0,BTC,-0.00529,...,86598.89,-1.339611,13.652679,-0.056911,-0.358936,0.141319,2024-11-13 06:30:00,67.931157,86626.538437,136.100990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29584,259800,2025-09-26 08:00:00,0.0260,65.17,6.4,0.0225,63.12,17.6,BTC,0.10559,...,86598.89,0.064127,0.299875,-0.452099,-0.332993,0.813112,2024-11-13 06:30:00,54.586601,95233.632187,64.712694
29585,259850,2025-09-26 08:00:00,0.0260,65.17,6.4,0.0225,63.12,17.6,BTC,0.10559,...,86598.89,0.064127,0.299875,-0.452099,-0.332993,0.813112,2024-11-13 06:30:00,54.586601,95233.632187,64.715102
29586,259900,2025-09-26 08:00:00,0.0260,65.17,6.4,0.0225,63.12,17.6,BTC,0.10559,...,86598.89,0.064127,0.299875,-0.452099,-0.332993,0.813112,2024-11-13 06:30:00,54.586601,95233.632187,64.717509
29587,259950,2025-09-26 08:00:00,0.0260,65.17,6.4,0.0225,63.12,17.6,BTC,0.10559,...,86598.89,0.064127,0.299875,-0.452099,-0.332993,0.813112,2024-11-13 06:30:00,54.586601,95233.632187,64.719916


# Plot Curves

In [17]:
import matplotlib.pyplot as plt

for exp in final_current.expirationTimestamp.unique():
    try:
        fig = go.Figure()

        final_exp_current = final_current   [final_current['expirationTimestamp'] == exp].sort_values('strike')
        hifi_exp_current  = hifi_current    [hifi_current ['expirationTimestamp'] == exp].sort_values('strike')
        bidIv             = hifi_exp_current['bidIv']
        askIv             = hifi_exp_current['askIv']

        fig.add_trace(go.Scatter(x=final_exp_current.strike, y=final_exp_current.sviIv, name="SVI current", legendrank=1, line=dict(color='green')))
        fig.add_trace(go.Scatter(x=hifi_exp_current .strike, y=bidIv, name='bid', mode='markers', marker_symbol="x-thin", marker_color="grey", marker_line_width=1, legendrank=3, visible='legendonly'))
        fig.add_trace(go.Scatter(x=hifi_exp_current .strike, y=askIv, name='ask', mode='markers', marker_symbol="x-thin", marker_color="grey", marker_line_width=1, legendrank=4, visible='legendonly'))

        fig.add_vline(x=final_exp_current['underlyingPrice_svi'].values[0], line_dash='dashdot', line_width=2, line_color="green")

        final_exp_current.replace('C', 'call', inplace=True)
        final_exp_current.replace('P', 'put',  inplace=True)

        fig.add_annotation(
            text      = str(pd.to_datetime(final_exp_current.timestamp_hifi.values[0])),
            opacity   = 0.5,
            xref      = "paper",
            yref      = "paper",
            x         = 1,
            y         = 0.05,
            showarrow = False,
            font      = dict(color='green')
        )

        fig.update_layout(
            title       = currency + " " + str(exp)[0:10] + " $" + str(round(final_exp_current['underlyingPrice_svi'].values[0], 2)),
            title_x     = 0.50,
            xaxis_title = "",
            yaxis_title = "iv",
            legend      = dict(orientation="h", yanchor="bottom", y=-0.20, xanchor="center", x=0.5),
            template    = 'plotly_white'
        )

        fig.layout.images = [dict(
            source  = AD_LOGO,
            xref    = "paper", yref="paper",
            x       = 0.96,
            y       = 1.05,
            sizex   = 0.13,
            sizey   = 0.13,
            xanchor = "center",
            yanchor = "bottom"
        )]

        fig.update_xaxes(showline=True, showgrid=True, tickmode='auto')

        fig.show()

    except Exception as e:
        print(f"Error: {e}")